<a href="https://colab.research.google.com/github/pragneshrana/WorkSample/blob/main/AutomationControl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# R&D @ [Nishtha Automation](http://www.nishthaindia.com/)

- Nishtha has developed a machine designed for processing raw diamonds. This machine is equipped to perform four primary processes: Table Sawing, Crowning, Bruiting (Girdle Outline), Cone Blocking, and additional facet processes.

- The primary objective of the project was to create an analytical/machine learning model capable of predicting potential diamond failures based on specific parameters.

- Another key goal was to implement a predictive model to control machine vibrations, with the aim of preventing failures by anticipating and mitigating excessive vibration levels.

#### Sample of Diamonds parameter data

In [3]:
import numpy as np
import pandas as pd

# Function to generate dummy dataset for diamond parameters
def generate_dummy_diamond_data(num_samples=1000):
    # Carat Weight (random values between 0.2 and 3.0)
    carat_weight = np.random.uniform(0.2, 3.0, num_samples)

    # Cut (random choice from common cut grades)
    cut_grades = ['Excellent', 'Very Good', 'Good', 'Fair', 'Poor']
    cut = np.random.choice(cut_grades, num_samples)

    # Color (random choice from common color grades)
    color_grades = ['D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M']
    color = np.random.choice(color_grades, num_samples)

    # Clarity (random choice from common clarity grades)
    clarity_grades = ['FL', 'IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1', 'I2', 'I3']
    clarity = np.random.choice(clarity_grades, num_samples)

    # Shape (random choice from common diamond shapes)
    diamond_shapes = ['Round', 'Princess', 'Emerald', 'Pear', 'Oval', 'Marquise', 'Heart', 'Radiant', 'Asscher']
    shape = np.random.choice(diamond_shapes, num_samples)

    # Hardness (assigning a constant value for diamond hardness)
    hardness = '10 (Mohs Scale)'

    # Purity and Type (random choices for purity and type)
    purity_types = ['Type Ia', 'Type Ib', 'Type IIa', 'Type IIb']
    purity = np.random.choice(purity_types, num_samples)

    diamond_types = ['Natural', 'Synthetic']
    diamond_type = np.random.choice(diamond_types, num_samples)

    # Table Size Percentage (random values between 50 and 70)
    table_size_percentage = np.random.uniform(50, 70, num_samples)

    # Crown Angle (random values between 25 and 35 degrees)
    crown_angle = np.random.uniform(25, 35, num_samples)

    # Crown Height Percentage (random values between 10 and 20)
    crown_height_percentage = np.random.uniform(10, 20, num_samples)

    # Pavilion Angle (random values between 40 and 45 degrees)
    pavilion_angle = np.random.uniform(40, 45, num_samples)

    # Pavilion Depth Percentage (random values between 40 and 45)
    pavilion_depth_percentage = np.random.uniform(40, 45, num_samples)

    # Girdle Thickness (random values between 1 and 3)
    girdle_thickness = np.random.uniform(1, 3, num_samples)

    # Culet Size (random values between 0 and 0.3)
    culet_size = np.random.uniform(0, 0.3, num_samples)

    # Impurity details
    impurity_types = np.random.choice(['Nitrogen', 'Boron', 'Hydrogen', 'Phosphorus', 'Sulfur'], num_samples)
    impurity_count = np.random.randint(0, 5, num_samples)  # Number of impurities
    impurity_size = np.random.uniform(0.01, 0.5, num_samples)  # Impurity size in carats
    impurity_location = np.random.choice(['Surface', 'Internal'], num_samples)  # Location of impurities

    # Create a DataFrame
    data = pd.DataFrame({
        'Weight': carat_weight,
        'Cut': cut,
        'Color': color,
        'Clarity': clarity,
        'Shape': shape,
        'Hardness': hardness,
        'Purity': purity,
        'Type': diamond_type,
        'Table Size Percentage': table_size_percentage,
        'Crown Angle': crown_angle,
        'Crown Height Percentage': crown_height_percentage,
        'Pavilion Angle': pavilion_angle,
        'Pavilion Depth Percentage': pavilion_depth_percentage,
        'Girdle Thickness': girdle_thickness,
        'Culet Size': culet_size,
        'Impurity Type': impurity_types,
        'Impurity Count': impurity_count,
        'Impurity Size (carats)': impurity_size,
        'Impurity Location': impurity_location
    })

    return data

# Example usage
diamond_data = generate_dummy_diamond_data(num_samples=1000)
diamond_data.head()


,Weight,Cut,Color,Clarity,Shape,Hardness,Purity,Type,Table Size Percentage,Crown Angle,Crown Height Percentage,Pavilion Angle,Pavilion Depth Percentage,Girdle Thickness,Culet Size,Impurity Type,Impurity Count,Impurity Size (carats),Impurity Location
0,2.007031,Fair,K,VVS1,Pear,10 (Mohs Scale),Type IIa,Synthetic,54.906552,27.964886,10.768819,43.350082,43.448190,1.069045,0.188065,Boron,4,0.057388,Internal
1,1.013267,Excellent,K,I3,Round,10 (Mohs Scale),Type IIa,Synthetic,60.462752,26.359587,15.900792,44.968142,44.716493,1.455632,0.187334,Boron,0,0.330855,Internal
2,2.555651,Poor,J,I3,Radiant,10 (Mohs Scale),Type IIb,Natural,69.043965,34.206526,10.066861,44.207684,41.206923,1.172980,0.137991,Hydrogen,0,0.208613,Surface
3,0.975110,Very Good,G,I2,Round,10 (Mohs Scale),Type Ib,Synthetic,61.055885,33.709664,15.149353,43.758199,43.610459,2.830696,0.191827,Hydrogen,0,0.435513,Internal
4,1.185262,Poor,L,VVS2,Heart,10 (Mohs Scale),Type IIb,Synthetic,64.807916,33.214436,11.104017,40.440241,40.725551,2.560669,0.284667,Nitrogen,3,0.311333,Internal
